# EDA


- The os module has a perfect method to list files in a directory.
- Pandas json normalize could work here but is not necessary to convert the JSON data to a dataframe.
- You may need a nested for-loop to access each sale!
- We've put a lot of time into creating the structure of this repository, and it's a good example for future projects.  In the file functions_variables.py, there is an example function that you can import and use.  If you have any variables, functions or classes that you want to make, they can be put in the functions_variables.py file and imported into a notebook.  Note that only .py files can be imported into a notebook. If you want to import everything from a .py file, you can use the following:
```python
from functions_variables import *
```
If you just import functions_variables, then each object from the file will need to be prepended with "functions_variables"\
Using this .py file will keep your notebooks very organized and make it easier to reuse code between notebooks.

In [182]:
# (this is not an exhaustive list of libraries)
import pandas as pd
import numpy as np
import os
import json
import re
from pprint import pprint
from functions_variables import encode_tags
from functions_variables import encode_primary_photo
from functions_variables import encode_source
from functions_variables import extract_city_state

## Data Importing

In [183]:
import os
import json
import pandas as pd

# Directory containing JSON files
directory = r"C:\Users\lai29\OneDrive - UBC\Documents\GitHub\Midterm-project\data"

# Initialize an empty list to store data
data_list = []

# Loop through each JSON file in the directory
for filename in os.listdir(directory):
    if filename.endswith(".json"):
        file_path = os.path.join(directory, filename)
        try:
            with open(file_path, "r", encoding="utf-8") as file:
                data = json.load(file)  # Load JSON data

            # Check if expected keys exist before accessing them
            if isinstance(data, dict) and "data" in data and isinstance(data["data"], dict) and "results" in data["data"]:
                for record in data["data"]["results"]:
                    if isinstance(record, dict):  # Ensure record is a dictionary before processing
                        flat_record = record.copy()

                        # Extract state and city from location
                        flat_record["state"] = None
                        flat_record["city"] = None

                        if "location" in record and isinstance(record["location"], dict):
                            address = record["location"].get("address", {})
                            flat_record["state"] = address.get("state", None)
                            flat_record["city"] = address.get("city", None)

                        # Flatten description
                        if "description" in record and isinstance(record["description"], dict):
                            flat_record.update(record["description"])
                            del flat_record["description"]

                        # Flatten products (dictionary)
                        if "products" in record and isinstance(record["products"], dict):
                            flat_record.update(pd.json_normalize(record["products"]).to_dict(orient="records")[0])
                            del flat_record["products"]

                        # Append flattened record to list
                        data_list.append(flat_record)

        except json.JSONDecodeError as e:
            print(f"Error decoding JSON in file {filename}: {e}")

# Convert list of dictionaries into a DataFrame
df_combined = pd.DataFrame(data_list)

# Display or save the combined DataFrame
print(df_combined[["state", "city"]].head())  # View extracted state and city columns
df_combined.to_csv("combined_data.csv", index=False)  # Save as CSV


    state    city
0  Alaska  Juneau
1  Alaska  Juneau
2  Alaska  Juneau
3  Alaska  Juneau
4  Alaska  Juneau


## Data Cleaning and Wrangling

At this point, ensure that you have all sales in a dataframe.
- Take a quick look at your data (i.e. `.info()`, `.describe()`) - what do you see?
- Is each cell one value, or do some cells have lists?
- What are the data types of each column?
- Some sales may not actually include the sale price (target).  These rows should be dropped.
- There are a lot of NA/None values.  Should these be dropped or replaced with something?
    - You can drop rows or use various methods to fills NA's - use your best judgement for each column 
    - i.e. for some columns (like Garage), NA probably just means no Garage, so 0
- Drop columns that aren't needed
    - Don't keep the list price because it will be too close to the sale price. Assume we want to predict the price of houses not yet listed

In [184]:
df_combined.info()
df_combined.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8159 entries, 0 to 8158
Data columns (total 41 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   primary_photo         7403 non-null   object 
 1   last_update_date      8125 non-null   object 
 2   source                7752 non-null   object 
 3   tags                  7638 non-null   object 
 4   permalink             8159 non-null   object 
 5   status                8159 non-null   object 
 6   list_date             7752 non-null   object 
 7   open_houses           0 non-null      object 
 8   branding              8159 non-null   object 
 9   list_price            7721 non-null   float64
 10  lead_attributes       8159 non-null   object 
 11  property_id           8159 non-null   object 
 12  photos                7403 non-null   object 
 13  flags                 8159 non-null   object 
 14  community             5 non-null      object 
 15  virtual_tours        

,list_price,price_reduced_amount,year_built,baths_3qtr,sold_price,baths_full,baths_half,lot_sqft,sqft,baths,garage,stories,beds,products
count,7.721000e+03,2.484000e+03,7316.000000,566.000000,6.716000e+03,7311.000000,2281.000000,6.991000e+03,7323.000000,7980.000000,4448.000000,6260.000000,7504.000000,0.0
mean,4.341582e+05,2.442704e+04,1968.916074,1.247350,4.126050e+05,1.887430,1.105655,2.510949e+05,1933.848559,2.131203,1.926709,1.567732,3.208289,NaN
std,5.514925e+05,7.162396e+04,35.096914,0.463482,6.994308e+05,0.862214,0.413340,5.823820e+06,1339.039206,1.175940,0.878766,0.730969,1.282732,NaN
min,1.000000e+00,1.000000e+02,1828.000000,1.000000,3.080000e+02,1.000000,1.000000,0.000000e+00,120.000000,0.000000,1.000000,1.000000,0.000000,NaN
25%,2.090000e+05,6.000000e+03,1950.000000,1.000000,1.910000e+05,1.000000,1.000000,4.953000e+03,1258.000000,1.000000,1.000000,1.000000,3.000000,NaN
50%,3.250000e+05,1.010000e+04,1975.000000,1.000000,3.140000e+05,2.000000,1.000000,7.841000e+03,1635.000000,2.000000,2.000000,1.000000,3.000000,NaN
75%,4.999000e+05,2.000000e+04,1997.000000,1.000000,4.700000e+05,2.000000,1.000000,1.263200e+04,2264.000000,3.000000,2.000000,2.000000,4.000000,NaN
max,1.250000e+07,2.015999e+06,2024.000000,3.000000,2.706500e+07,8.000000,5.000000,1.676624e+08,32106.000000,9.000000,11.000000,10.000000,12.000000,NaN


In [185]:
# load and concatenate data here
# drop or replace values as necessary

# Examine df_combined dtypes
df_combined.dtypes

# Create a list of categorical variables
categorical = df_combined.select_dtypes(include=['object']).columns

# Create a list of numerical variables
numerical = df_combined.select_dtypes(exclude=['object']).columns

print(categorical)
print(numerical)

Index(['primary_photo', 'last_update_date', 'source', 'tags', 'permalink',
       'status', 'list_date', 'open_houses', 'branding', 'lead_attributes',
       'property_id', 'photos', 'flags', 'community', 'virtual_tours',
       'other_listings', 'listing_id', 'location', 'state', 'city',
       'sold_date', 'name', 'sub_type', 'baths_1qtr', 'type', 'brand_name'],
      dtype='object')
Index(['list_price', 'price_reduced_amount', 'matterport', 'year_built',
       'baths_3qtr', 'sold_price', 'baths_full', 'baths_half', 'lot_sqft',
       'sqft', 'baths', 'garage', 'stories', 'beds', 'products'],
      dtype='object')


In [196]:
# Columns to drop
columns_to_drop = ['last_update_date','location','sold_date','branding','flags','open_houses','status','lead_attributes' ,'property_id', 'photos', 'listing_id', 'list_price','other_listings', 'community', 'products',
       'virtual_tours','name']

# Drop the unneeded columns
df_dropped = df_combined.drop(columns=columns_to_drop)



In [187]:
df_dropped.head()

,primary_photo,source,tags,permalink,list_date,price_reduced_amount,location,matterport,state,city,...,lot_sqft,sqft,baths,sub_type,baths_1qtr,garage,stories,beds,type,brand_name
0,{'href': 'https://ap.rdcpix.com/07097d34c98a59...,"{'plan_id': None, 'agents': [{'office_name': '...","[carport, community_outdoor_space, cul_de_sac,...",9453-Herbert-Pl_Juneau_AK_99801_M90744-30767,2023-06-29T21:16:25.000000Z,45000.0,"{'address': {'postal_code': '99801', 'state': ...",False,Alaska,Juneau,...,10454.0,1821.0,2.0,None,None,1.0,NaN,3.0,single_family,basic_opt_in
1,None,None,None,8477-Thunder-Mountain-Rd_Juneau_AK_99801_M9424...,None,NaN,"{'address': {'postal_code': '99801', 'state': ...",False,Alaska,Juneau,...,NaN,NaN,NaN,None,None,NaN,NaN,NaN,None,NaN
2,None,None,None,4515-Glacier-Hwy_Juneau_AK_99801_M94790-68516,None,NaN,"{'address': {'postal_code': '99801', 'state': ...",False,Alaska,Juneau,...,NaN,NaN,NaN,None,None,NaN,NaN,NaN,None,NaN
3,None,None,None,17850-Point-Stephens-Rd_Juneau_AK_99801_M98793...,None,NaN,"{'address': {'postal_code': '99801', 'state': ...",False,Alaska,Juneau,...,NaN,NaN,NaN,None,None,NaN,NaN,NaN,None,NaN
4,None,None,None,9951-Stephen-Richards-Memorial-Dr_Juneau_AK_99...,None,NaN,"{'address': {'postal_code': '99801', 'state': ...",False,Alaska,Juneau,...,NaN,NaN,NaN,None,None,NaN,NaN,NaN,None,NaN


In [201]:
#Drop NA values for sales Price

df_cleaned = df_dropped.dropna(subset=['sold_price'])

#Fill NA Cities with most frequent City for each State

df_cleaned['city'] = df_cleaned['city'].fillna(
    df_dropped.groupby('state')['city'].transform(lambda x: x.value_counts().idxmax() if not x.dropna().empty else None)
)

#Replace NA with 0 for baths_3qtr, baths_full, and baths_half, baths_1qtr,garage, stories, beds

df_cleaned = df_cleaned.fillna({'baths_3qtr': 0, 'baths_full': 0, 'baths_half': 0, 'baths_1qtr': 0,'garage': 0, 'stories': 0, 'beds': 0})

#replace NA in type, sub_type with 'other'
df_cleaned = df_cleaned.fillna({'type': 'other', 'sub_type': 'other'})

C:\Users\lai29\AppData\Local\Temp\ipykernel_29116\3506468924.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['city'] = df_cleaned['city'].fillna(
C:\Users\lai29\AppData\Local\Temp\ipykernel_29116\3506468924.py:13: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_cleaned = df_cleaned.fillna({'baths_3qtr': 0, 'baths_full': 0, 'baths_half': 0, 'baths_1qtr': 0,'garage': 0, 'stories': 0, 'beds': 0})


In [189]:
df_cleaned.head()

,primary_photo,source,tags,permalink,list_date,price_reduced_amount,location,matterport,state,city,...,lot_sqft,sqft,baths,sub_type,baths_1qtr,garage,stories,beds,type,brand_name
30,None,"{'plan_id': None, 'agents': [{'office_name': '...","[central_air, central_heat, community_outdoor_...",501-Moorfield-Ln_Montgomery_AL_36117_M88465-41030,2023-12-19T18:08:22.000000Z,NaN,"{'address': {'postal_code': '36117', 'state': ...",False,Alabama,Montgomery,...,11761.0,1478.0,2.0,other,0,2.0,1.0,3.0,single_family,essentials
31,{'href': 'https://ap.rdcpix.com/a5fe1391acf35d...,"{'plan_id': None, 'agents': [{'office_name': '...","[city_view, hardwood_floors, two_or_more_stori...",2326-Winona-Ave_Montgomery_AL_36107_M77279-81021,2023-11-10T14:20:36.000000Z,3000.0,"{'address': {'postal_code': '36107', 'state': ...",False,Alabama,Montgomery,...,6534.0,1389.0,2.0,other,0,1.0,2.0,4.0,single_family,essentials
32,None,"{'plan_id': None, 'agents': [{'office_name': '...","[carport, central_air, central_heat, dishwashe...",115-Lookout-Ridge-Rd_Montgomery_AL_36109_M7320...,2023-12-19T18:08:22.000000Z,NaN,"{'address': {'postal_code': '36109', 'state': ...",False,Alabama,Montgomery,...,17424.0,2058.0,2.0,other,0,0.0,1.0,3.0,single_family,essentials
33,{'href': 'https://ap.rdcpix.com/c41ec0e28b89fa...,"{'plan_id': None, 'agents': [{'office_name': '...","[carport, central_air, city_view, community_ou...",611-Glenmore-Rd_Montgomery_AL_36107_M72316-04965,2023-07-05T16:47:40.000000Z,9000.0,"{'address': {'postal_code': '36107', 'state': ...",False,Alabama,Montgomery,...,9712.0,1432.0,2.0,other,0,0.0,1.0,3.0,single_family,essentials
34,{'href': 'https://ap.rdcpix.com/1e4bac70e24f78...,"{'plan_id': None, 'agents': [{'office_name': '...","[city_view, single_story, big_yard, fenced_yar...",1819-E-Trinity-Blvd_Montgomery_AL_36106_M77006...,2023-10-29T17:55:28.000000Z,5000.0,"{'address': {'postal_code': '36106', 'state': ...",False,Alabama,Montgomery,...,10890.0,1804.0,2.0,other,0,0.0,1.0,3.0,single_family,basic_opt_in


### Dealing with Tags

Consider the fact that with tags, there are a lot of categorical variables.
- How many columns would we have if we OHE tags, city and state?
- Perhaps we can get rid of tags that have a low frequency.

In [202]:
# Encode Tags on df_cleaned
df_encoded = encode_tags(df_cleaned)

# Run encode_source function
df_encoded = encode_source(df_encoded)

# Encode Primary Photo on df_encoded
df_encoded = encode_primary_photo(df_encoded)

# Convert to datetime format
df_encoded['list_date'] = pd.to_datetime(df_encoded['list_date'], errors='coerce')

# Format as MM-DD-YYYY
df_encoded['list_date'] = df_encoded['list_date'].dt.strftime('%m-%d-%Y')

df_encoded.head()

,permalink,list_date,price_reduced_amount,matterport,state,city,year_built,baths_3qtr,sold_price,baths_full,...,['single_story',['single_story'],['two_or_more_stories',['two_or_more_stories'],['view',['washer_dryer',['wooded_land',agent,mls,has_primary_photo
30,501-Moorfield-Ln_Montgomery_AL_36117_M88465-41030,12-19-2023,NaN,False,Alabama,Montgomery,1998.0,0.0,129900.0,2.0,...,0,0,0,0,0,0,0,0,0,False
31,2326-Winona-Ave_Montgomery_AL_36107_M77279-81021,11-10-2023,3000.0,False,Alabama,Montgomery,1945.0,0.0,88500.0,2.0,...,0,0,0,0,0,0,0,0,0,True
32,115-Lookout-Ridge-Rd_Montgomery_AL_36109_M7320...,12-19-2023,NaN,False,Alabama,Montgomery,1969.0,0.0,145000.0,2.0,...,0,0,0,0,0,0,0,0,0,False
33,611-Glenmore-Rd_Montgomery_AL_36107_M72316-04965,07-05-2023,9000.0,False,Alabama,Montgomery,1955.0,0.0,65000.0,2.0,...,0,0,0,0,0,0,0,0,0,True
34,1819-E-Trinity-Blvd_Montgomery_AL_36106_M77006...,10-29-2023,5000.0,False,Alabama,Montgomery,1984.0,0.0,169000.0,2.0,...,0,0,0,0,0,0,0,0,0,True


### Dealing with Cities

- Sales will vary drastically between cities and states.  Is there a way to keep information about which city it is without OHE?
- Could we label encode or ordinal encode?  Yes, but this may have undesirable effects, giving nominal data ordinal values.
- What we can do is use our training data to encode the mean sale price by city as a feature (a.k.a. Target Encoding)
    - We can do this as long as we ONLY use the training data - we're using the available data to give us a 'starting guess' of the price for each city, without needing to encode city explicitly
- If you replace cities or states with numerical values (like the mean price), make sure that the data is split so that we don't leak data into the training selection. This is a great time to train test split. Compute on the training data, and join these values to the test data
- Note that you *may* have cities in the test set that are not in the training set. You don't want these to be NA, so maybe you can fill them with the overall mean

In [199]:
# perform train test split here
# Examine permalink for city and state info
##df_encoded['permalink'].head() ##Adding this to the JSON merge instead

# Run extract_city_state function
#df_encoded = extract_city_state(df_encoded)

# Remove the '-' separating multi word cities and replacing with ' '
df_encoded['city'] = df_encoded['city'].str.replace('-', ' ', regex=False)

df_encoded[['city','state']].head(-40)

# Save a copy of df_encoded csv
df_encoded.to_csv("encoded_data.csv", index=False)

df_encoded.head()

,permalink,list_date,price_reduced_amount,location,matterport,state,city,year_built,baths_3qtr,sold_price,...,['single_story',['single_story'],['two_or_more_stories',['two_or_more_stories'],['view',['washer_dryer',['wooded_land',agent,mls,has_primary_photo
30,501-Moorfield-Ln_Montgomery_AL_36117_M88465-41030,12-19-2023,NaN,"{'address': {'postal_code': '36117', 'state': ...",False,Alabama,Montgomery,1998.0,0.0,129900.0,...,0,0,0,0,0,0,0,0,0,False
31,2326-Winona-Ave_Montgomery_AL_36107_M77279-81021,11-10-2023,3000.0,"{'address': {'postal_code': '36107', 'state': ...",False,Alabama,Montgomery,1945.0,0.0,88500.0,...,0,0,0,0,0,0,0,0,0,True
32,115-Lookout-Ridge-Rd_Montgomery_AL_36109_M7320...,12-19-2023,NaN,"{'address': {'postal_code': '36109', 'state': ...",False,Alabama,Montgomery,1969.0,0.0,145000.0,...,0,0,0,0,0,0,0,0,0,False
33,611-Glenmore-Rd_Montgomery_AL_36107_M72316-04965,07-05-2023,9000.0,"{'address': {'postal_code': '36107', 'state': ...",False,Alabama,Montgomery,1955.0,0.0,65000.0,...,0,0,0,0,0,0,0,0,0,True
34,1819-E-Trinity-Blvd_Montgomery_AL_36106_M77006...,10-29-2023,5000.0,"{'address': {'postal_code': '36106', 'state': ...",False,Alabama,Montgomery,1984.0,0.0,169000.0,...,0,0,0,0,0,0,0,0,0,True


## Extra Data - STRETCH

> This doesn't need to be part of your Minimum Viable Product (MVP). We recommend you write a functional, basic pipeline first, then circle back and join new data if you have time

> If you do this, try to write your downstream steps in a way it will still work on a dataframe with different features!

- You're not limited to just using the data provided to you. Think/ do some research about other features that might be useful to predict housing prices. 
- Can you import and join this data? Make sure you do any necessary preprocessing and make sure it is joined correctly.
- Example suggestion: could mortgage interest rates in the year of the listing affect the price? 

In [203]:
# import, join and preprocess new data here
# Read city demographic data in df_city_data
df_city_data = pd.read_csv("us-cities-demographics.csv", sep=";")

# Pivot the df_city_data so 'Race' becomes columns and 'Count' is the value
df_city_pivot = df_city_data.pivot_table(
    index=['City', 'State Code'], 
    columns='Race', 
    values='Count', 
    aggfunc='sum'  # Ensure counts are summed if duplicates exist
).reset_index()

# Rename race columns for cleaner names
df_city_pivot.columns.name = None  # Remove MultiIndex column name
df_city_pivot = df_city_pivot.rename(columns=lambda x: x.replace(" ", "_") if isinstance(x, str) else x)  # Replace spaces with underscores

# Merge transformed df_city_pivot with df_encoded
df_merged = df_encoded.merge(
    df_city_pivot,
    left_on=['city', 'state'],
    right_on=['City', 'State_Code'],
    how='left'
)

# Remove Race and City columns from df_city_data
df_city_data_cleaned = df_city_data.drop(columns=['Race', 'Count'])

# Merge df_merged with df_city_data_cleaned
df_merged = df_merged.merge(
    df_city_data_cleaned,
    left_on=['city','state'],
    right_on=['City','State Code'],
    how='left'
)

# Drop unnecessary columns after merging
df_merged.drop(columns=['City_y','City_x','State_Code', 'State Code'], inplace=True)

# Drop duplicate rows
df_merged = df_merged.drop_duplicates()

# Display first few rows to check the result
df_merged.head()

,permalink,list_date,price_reduced_amount,matterport,state,city,year_built,baths_3qtr,sold_price,baths_full,...,Hispanic_or_Latino,White,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size
0,501-Moorfield-Ln_Montgomery_AL_36117_M88465-41030,12-19-2023,NaN,False,Alabama,Montgomery,1998.0,0.0,129900.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2326-Winona-Ave_Montgomery_AL_36107_M77279-81021,11-10-2023,3000.0,False,Alabama,Montgomery,1945.0,0.0,88500.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,115-Lookout-Ridge-Rd_Montgomery_AL_36109_M7320...,12-19-2023,NaN,False,Alabama,Montgomery,1969.0,0.0,145000.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,611-Glenmore-Rd_Montgomery_AL_36107_M72316-04965,07-05-2023,9000.0,False,Alabama,Montgomery,1955.0,0.0,65000.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1819-E-Trinity-Blvd_Montgomery_AL_36106_M77006...,10-29-2023,5000.0,False,Alabama,Montgomery,1984.0,0.0,169000.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [167]:
# Save a copy of df_merged csv
df_merged.to_csv("merged_data.csv", index=False)

In [204]:
df_merged.head()

,permalink,list_date,price_reduced_amount,matterport,state,city,year_built,baths_3qtr,sold_price,baths_full,...,Hispanic_or_Latino,White,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size
0,501-Moorfield-Ln_Montgomery_AL_36117_M88465-41030,12-19-2023,NaN,False,Alabama,Montgomery,1998.0,0.0,129900.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2326-Winona-Ave_Montgomery_AL_36107_M77279-81021,11-10-2023,3000.0,False,Alabama,Montgomery,1945.0,0.0,88500.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,115-Lookout-Ridge-Rd_Montgomery_AL_36109_M7320...,12-19-2023,NaN,False,Alabama,Montgomery,1969.0,0.0,145000.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,611-Glenmore-Rd_Montgomery_AL_36107_M72316-04965,07-05-2023,9000.0,False,Alabama,Montgomery,1955.0,0.0,65000.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1819-E-Trinity-Blvd_Montgomery_AL_36106_M77006...,10-29-2023,5000.0,False,Alabama,Montgomery,1984.0,0.0,169000.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [205]:
#Adding the cost of living dataset
cost_of_living_dataset = pd.read_csv("cost-of-living-index-by-state-2024.csv",sep=",")

In [206]:
#Merging the cost of living dataset to the merged dataset
cost_living_merged = pd.merge(df_merged, cost_of_living_dataset, on="state")

In [207]:
# Cleaning up column names
cost_living_merged.columns = (
    cost_living_merged.columns.str.strip() # Remove leading/trailing spaces
    .str.replace(r"[\[\]']", "", regex=True)  # Remove brackets and quotes
    .str.replace(" ", "_")    # Replace spaces with underscores
    .str.lower()              # Convert to lowercase
)

# Display cleaned column names
print(cost_living_merged.columns.tolist())

['permalink', 'list_date', 'price_reduced_amount', 'matterport', 'state', 'city', 'year_built', 'baths_3qtr', 'sold_price', 'baths_full', 'baths_half', 'lot_sqft', 'sqft', 'baths', 'sub_type', 'baths_1qtr', 'garage', 'stories', 'beds', 'type', 'brand_name', 'baseball', 'basement', 'basement', 'basketball', 'basketball', 'basketball_court', 'beach', 'beach', 'beautiful_backyard', 'beautiful_backyard', 'big_bathroom', 'big_bathroom', 'big_lot', 'big_lot', 'big_yard', 'big_yard', 'boat_dock', 'cathedral_ceiling', 'central_air', 'central_heat', 'city_view', 'clubhouse', 'clubhouse', 'coffer_ceiling', 'coffer_ceiling', 'community_boat_facilities', 'community_center', 'community_center', 'community_clubhouse', 'community_elevator', 'community_golf', 'community_gym', 'community_horse_facilities', 'community_outdoor_space', 'community_park', 'community_security_features', 'community_spa_or_hot_tub', 'community_swimming_pool', 'community_tennis_court', 'corner_lot', 'corner_lot', 'courtyard_ent

In [208]:
for col in cost_living_merged.columns:
    print(col)

permalink
list_date
price_reduced_amount
matterport
state
city
year_built
baths_3qtr
sold_price
baths_full
baths_half
lot_sqft
sqft
baths
sub_type
baths_1qtr
garage
stories
beds
type
brand_name
baseball
basement
basement
basketball
basketball
basketball_court
beach
beach
beautiful_backyard
beautiful_backyard
big_bathroom
big_bathroom
big_lot
big_lot
big_yard
big_yard
boat_dock
cathedral_ceiling
central_air
central_heat
city_view
clubhouse
clubhouse
coffer_ceiling
coffer_ceiling
community_boat_facilities
community_center
community_center
community_clubhouse
community_elevator
community_golf
community_gym
community_horse_facilities
community_outdoor_space
community_park
community_security_features
community_spa_or_hot_tub
community_swimming_pool
community_tennis_court
corner_lot
corner_lot
courtyard_entry
cul_de_sac
den_or_office
detached_guest_house
dining_room
disability_features
dishwasher
dual_master_bedroom
efficient
efficient
elevator
elevator
energy_efficient
ensuite
ensuite
expos

In [209]:
# Creating a list of merged columns to drop
additional_cols = ['list_date', 'location','costoflivingindexmisccostsindex', 'costoflivingindexhealthcostsindex','costoflivingindextransportationcostsindex', 
                   'costoflivingindexutilitycostsindex','costoflivingindexmisccostsindex', 
                   'costoflivingindextransportationcostsindex', 'costoflivingindexutilitycostsindex','number_of_veterans', 'foreign-born', 'hispanic_or_latino', 'white', 'female_population', 'male_population',
                   'american_indian_and_alaska_native','asian','black_or_african-american',
                   'horse_property','horse_stables', 'golf_course_lot_or_frontage', 'golf_course_view','low_hoa','solar_panels','tennis_court','views', #removing columns with potential multicolinearity
                   'lake_view','pond', #Removing these due to water_view column
                   'rv_parking', 'river_access','basketball','efficient', #Removing Columns with multiple names e.g view vs views
                   'golf_course_view','golf_course_lot_or_frontage'
                   ]

#Dropping all unnecessary columns
cost_living_merged_dropped = cost_living_merged.drop(columns=additional_cols, errors = 'ignore')
cost_living_merged_dropped = cost_living_merged_dropped.loc[:, ~cost_living_merged_dropped.columns.duplicated()] #Dropping duplicate columns

In [210]:
#Print list of new columns
for col in cost_living_merged_dropped.columns:
    print(col)

permalink
price_reduced_amount
matterport
state
city
year_built
baths_3qtr
sold_price
baths_full
baths_half
lot_sqft
sqft
baths
sub_type
baths_1qtr
garage
stories
beds
type
brand_name
baseball
basement
basketball_court
beach
beautiful_backyard
big_bathroom
big_lot
big_yard
boat_dock
cathedral_ceiling
central_air
central_heat
city_view
clubhouse
coffer_ceiling
community_boat_facilities
community_center
community_clubhouse
community_elevator
community_golf
community_gym
community_horse_facilities
community_outdoor_space
community_park
community_security_features
community_spa_or_hot_tub
community_swimming_pool
community_tennis_court
corner_lot
courtyard_entry
cul_de_sac
den_or_office
detached_guest_house
dining_room
disability_features
dishwasher
dual_master_bedroom
elevator
energy_efficient
ensuite
exposed_brick
family_room
farm
fenced_courtyard
fenced_yard
fireplace
first_floor_master_bedroom
fixer_upper
floor_plan
forced_air
front_porch
fruit_trees
furniture
game_room
garage_1_or_more

In [211]:
#Encode - Could also hot encode this 

# Define the categorization function
def categorize_cost_of_living(score):
    if score < 100:
        return 'Low'
    elif 100 <= score <= 120:
        return 'Average'
    elif 120 < score <= 150:
        return 'High'
    else:
        return 'Very High'

# Apply the categorization function to COL columns
cost_living_merged_dropped['cost_of_living_overall'] = cost_living_merged_dropped['costoflivingindex2023'].apply(categorize_cost_of_living)
cost_living_merged_dropped['cost_of_living_grocery'] = cost_living_merged_dropped['costoflivingindexgrocerycostsindex'].apply(categorize_cost_of_living)
cost_living_merged_dropped['cost_of_living_housing'] = cost_living_merged_dropped['costoflivingindexhousingcostsindex'].apply(categorize_cost_of_living)

# print new data
cost_living_merged_dropped.head()



,permalink,price_reduced_amount,matterport,state,city,year_built,baths_3qtr,sold_price,baths_full,baths_half,...,has_primary_photo,median_age,total_population,average_household_size,costoflivingindex2023,costoflivingindexgrocerycostsindex,costoflivingindexhousingcostsindex,cost_of_living_overall,cost_of_living_grocery,cost_of_living_housing
0,501-Moorfield-Ln_Montgomery_AL_36117_M88465-41030,NaN,False,Alabama,Montgomery,1998.0,0.0,129900.0,2.0,0.0,...,False,NaN,NaN,NaN,88.8,96.4,70.4,Low,Low,Low
1,2326-Winona-Ave_Montgomery_AL_36107_M77279-81021,3000.0,False,Alabama,Montgomery,1945.0,0.0,88500.0,2.0,0.0,...,True,NaN,NaN,NaN,88.8,96.4,70.4,Low,Low,Low
2,115-Lookout-Ridge-Rd_Montgomery_AL_36109_M7320...,NaN,False,Alabama,Montgomery,1969.0,0.0,145000.0,2.0,0.0,...,False,NaN,NaN,NaN,88.8,96.4,70.4,Low,Low,Low
3,611-Glenmore-Rd_Montgomery_AL_36107_M72316-04965,9000.0,False,Alabama,Montgomery,1955.0,0.0,65000.0,2.0,0.0,...,True,NaN,NaN,NaN,88.8,96.4,70.4,Low,Low,Low
4,1819-E-Trinity-Blvd_Montgomery_AL_36106_M77006...,5000.0,False,Alabama,Montgomery,1984.0,0.0,169000.0,2.0,0.0,...,True,NaN,NaN,NaN,88.8,96.4,70.4,Low,Low,Low


In [212]:
#Importing us census bureau regions and divisions 
region_data = pd.read_csv(r'us census bureau regions and divisions (1).csv')
region_data.head()

,State,State Code,Region,Division
0,Alaska,AK,West,Pacific
1,Alabama,AL,South,East South Central
2,Arkansas,AR,South,West South Central
3,Arizona,AZ,West,Mountain
4,California,CA,West,Pacific


In [213]:
cost_living_merged_dropped.head()

,permalink,price_reduced_amount,matterport,state,city,year_built,baths_3qtr,sold_price,baths_full,baths_half,...,has_primary_photo,median_age,total_population,average_household_size,costoflivingindex2023,costoflivingindexgrocerycostsindex,costoflivingindexhousingcostsindex,cost_of_living_overall,cost_of_living_grocery,cost_of_living_housing
0,501-Moorfield-Ln_Montgomery_AL_36117_M88465-41030,NaN,False,Alabama,Montgomery,1998.0,0.0,129900.0,2.0,0.0,...,False,NaN,NaN,NaN,88.8,96.4,70.4,Low,Low,Low
1,2326-Winona-Ave_Montgomery_AL_36107_M77279-81021,3000.0,False,Alabama,Montgomery,1945.0,0.0,88500.0,2.0,0.0,...,True,NaN,NaN,NaN,88.8,96.4,70.4,Low,Low,Low
2,115-Lookout-Ridge-Rd_Montgomery_AL_36109_M7320...,NaN,False,Alabama,Montgomery,1969.0,0.0,145000.0,2.0,0.0,...,False,NaN,NaN,NaN,88.8,96.4,70.4,Low,Low,Low
3,611-Glenmore-Rd_Montgomery_AL_36107_M72316-04965,9000.0,False,Alabama,Montgomery,1955.0,0.0,65000.0,2.0,0.0,...,True,NaN,NaN,NaN,88.8,96.4,70.4,Low,Low,Low
4,1819-E-Trinity-Blvd_Montgomery_AL_36106_M77006...,5000.0,False,Alabama,Montgomery,1984.0,0.0,169000.0,2.0,0.0,...,True,NaN,NaN,NaN,88.8,96.4,70.4,Low,Low,Low


In [214]:
df_merged_region = pd.merge(cost_living_merged_dropped, region_data, left_on=['state'], right_on =['State'])
df_merged_region.drop(columns=['State', 'State Code'], inplace=True)

In [215]:
df_merged_region.head()

,permalink,price_reduced_amount,matterport,state,city,year_built,baths_3qtr,sold_price,baths_full,baths_half,...,total_population,average_household_size,costoflivingindex2023,costoflivingindexgrocerycostsindex,costoflivingindexhousingcostsindex,cost_of_living_overall,cost_of_living_grocery,cost_of_living_housing,Region,Division
0,501-Moorfield-Ln_Montgomery_AL_36117_M88465-41030,NaN,False,Alabama,Montgomery,1998.0,0.0,129900.0,2.0,0.0,...,NaN,NaN,88.8,96.4,70.4,Low,Low,Low,South,East South Central
1,2326-Winona-Ave_Montgomery_AL_36107_M77279-81021,3000.0,False,Alabama,Montgomery,1945.0,0.0,88500.0,2.0,0.0,...,NaN,NaN,88.8,96.4,70.4,Low,Low,Low,South,East South Central
2,115-Lookout-Ridge-Rd_Montgomery_AL_36109_M7320...,NaN,False,Alabama,Montgomery,1969.0,0.0,145000.0,2.0,0.0,...,NaN,NaN,88.8,96.4,70.4,Low,Low,Low,South,East South Central
3,611-Glenmore-Rd_Montgomery_AL_36107_M72316-04965,9000.0,False,Alabama,Montgomery,1955.0,0.0,65000.0,2.0,0.0,...,NaN,NaN,88.8,96.4,70.4,Low,Low,Low,South,East South Central
4,1819-E-Trinity-Blvd_Montgomery_AL_36106_M77006...,5000.0,False,Alabama,Montgomery,1984.0,0.0,169000.0,2.0,0.0,...,NaN,NaN,88.8,96.4,70.4,Low,Low,Low,South,East South Central


In [217]:
# Look for cities missing demographic data
missing_population_cities = df_merged_region[df_merged_region['total_population'].isna()]['city'].unique()
for col in missing_population_cities:
    print(col)

Montgomery
Little Rock
Mabelvale
Alexander
Tolleson
Phoenix
Glendale
Cave Creek
Sacramento
Antelope
Elk Grove
Denver
Hartford
Dover
Tallahassee
Atlanta
Sandy Springs
Hapeville
Honolulu
Des Moines
Boone
Springfield
Indianapolis
Lawrence
Frankfort
Boston
Annapolis
Lansing
Saint Paul
Shoreview
Arden Hills
Little Canada
Raleigh
Lincoln
Concord
Trenton
Hamilton
Lawrenceville
Robbinsville
Carson City
Albany
Menands
Loudonville
Guilderland
Galloway
Columbus
Canal Winchester
Dublin
Grove City
Westerville
Reynoldsburg
Oklahoma City
Edmond
Yukon
Piedmont
Mustang
Midwest City
Spencer
Salem
Harrisburg
Providence
North Providence
Columbia
Garnett
Elgin
Hermitage
Old Hickory
Nashville
Antioch
Pegram
Joelton
Richmond
Henrico
Ampthill
Highland Springs
Olympia
Tumwater
Madison
Verona
McFarland
Charleston
Cross Lanes
South Charleston
Charlton Heights


In [ ]:
# Save a copy of updated df_merged csv
df_merged_region.to_csv("merged_data_region.csv", index=False)

## EDA/ Visualization

Remember all of the EDA that you've been learning about?  Now is a perfect time for it!
- Look at distributions of numerical variables to see the shape of the data and detect outliers.    
    - Consider transforming very skewed variables
- Scatterplots of a numerical variable and the target go a long way to show correlations.
- A heatmap will help detect highly correlated features, and we don't want these.
    - You may have too many features to do this, in which case you can simply compute the most correlated feature-pairs and list them
- Is there any overlap in any of the features? (redundant information, like number of this or that room...)

In [ ]:
# perform EDA here

#Heat Table
#Correlation Table
#Scatter Plots for Sale Price


## Scaling and Finishing Up

Now is a great time to scale the data and save it once it's preprocessed.
- You can save it in your data folder, but you may want to make a new `processed/` subfolder to keep it organized